# ANPR API comuni

## Creazione dataset per API

### Abstract
Esporra su DAF (?) un’API che fornisca le informazioni di base sui comuni italiani.
I dati di base contenenti l’elenco dei comuni sono quelli che fornisce il Ministero dell’Interno sul sito di ANPR. A corollario si integrano altre informazioni di utilità sui comuni italiani che provengono da diversi fonti dati che fornisce ISTAT.

### Note
Chiarire bene con Mirko le info del dataset di ANPR (comuni cessati, codice...) e come considerare ogni record: nella prima implementazione vado in join su codice istat . capire come raccordare i 19.000 record di ANPR con ISTAT (ad oggi su join codice ISTAT)

Stabilire la frequenza di aggiornamento del dataset considerando che il dataset di partenza (in formato Excel) ANPR viene pubblicato ogni giorno e contiene circa 19.000 record.

Verificare con Mirko se è già presente un flusso verso il team oppure dobbiamo implementarne uno ad hoc. Anche se la pubblicazione è giornaliera i dati sui comuni variano lentamente nel tempo.
i dataset con i dati a corollario di ISTAT sono sui comuni esistenti (poco meno di 8000), non vengono rilasciato con una frequenza prefissata, verranno quindi controllati periodicamente


### Dataset coinvolti:

#### ANPR: Archivio storico dei comuni

- Comuni subentrati
https://www.anpr.interno.it/portale/documents/20182/50186/Tabella_45+Comuni+subentrati+31052018.xlsx/3c699246-f351-499d-950e-244c794ee39b

- Tabelle di riferimento
https://www.anpr.interno.it/portale/tabelle-di-riferimento

Archivio storico dei comuni (01-06-2018) - xls
formato Excel -XSL

frequenza di aggiornamento?

definire flusso in DAF?

individuare persona di riferimento, sentire Mirko

##### Campi:

ID	

DATAISTITUZIONE	

DATACESSAZIONE	

CODISTAT (alfanumerico)

CODCATASTALE (rimane lo stesso anche quando cambia il codice istat?)	

DENOMINAZIONE_IT	

DENOMTRASLITTERATA	

ALTRADENOMINAZIONE	

ALTRADENOMTRASLITTERATA	

IDPROVINCIA	

IDREGIONE	

IDPREFETTURA	

STATO	

SIGLAPROVINCIA	

FONTE	

DATAULTIMOAGG	

COD_DENOM


#### ISTAT: 

 - Info geografiche reperite da "ARCHIVIO DEI CONFINI DELLE UNITÀ AMMINISTRATIVE A FINI STATISTICI:
 
 Comuni_2016.geojson (convertito shp)
https://www.istat.it/it/archivio/124086

- PERMALINK COMUNI Italiani https://www.istat.it/storage/codici-unita-amministrative/Elenco-comuni-italiani.csv
    
- PRINCIPALI STATISTICHE GEOGRAFICHE SUI COMUNI
https://www.istat.it/it/archivio/156224

In particolare sono integrati i seguenti dataset:

Classificazioni statistiche (zip) reperibile al link:
https://www.istat.it/it/files//2015/04/Classificazioni-statistiche.zip

Il file utilizzato è quello con la data più recente: "Classificazioni statistiche_comuni_17_02_2018.xls"

I campi contenuti sono:




Informazioni geografiche sui comuni:

poligono

latitudine e longitudine centroide


### Funzionalità’ API:

Il dataset di ANPR unito a dataset di ISTAT su campo codice ISTAT


* ricerca comune
* ricerca provincia: ritorna record con comuni provincia
* ricerca regione: ritorna record con comuni della regione

* next: dato una latitudine e longitudine ritorna comune di appartenenza


Da smarcare:

	questione comuni con serie storica

#### Utili:
Vedere post su forum per confronto:

* https://forum.italia.it/t/api-per-lelenco-dei-comuni-italiani/1349/18
* https://github.com/italia/anpr/issues/331
* https://github.com/teamdigitale/dashboards-script/tree/master/anpr
 

    

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime

from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #setta l'esecuzione di più statement in unica cella
pd.set_option('display.max_colwidth', -1) 
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 200) 

### ANPR: elenco comuni

Da notare che i due campi data sono in formato diverso (!)
es. 
* DATACESSAZIONE: 1861-03-17 e 13/11/1924 e 9999-12-31 00:00:00: 

> I valori uguali a 9999-12-31 indicano i comuni non cessati

* DATAISTITUZIONE 1948-12-05

Per capire a parità di nome quali sono gli attuali è corretto filtrare su:
DATACESSAZIONE = 31/12/9999	

In [2]:
anpr_comuni = pd.read_excel("./data/tabella 3 archivio comuni 20180601.xlsx", dtype={'CODISTAT':'str'})
#  alfanumerico poichè il file con le geo ha codice istat numerico

In [3]:
anpr_comuni.dtypes

ID                         int64  
DATAISTITUZIONE            object 
DATACESSAZIONE             object 
CODISTAT                   object 
CODCATASTALE               object 
DENOMINAZIONE_IT           object 
DENOMTRASLITTERATA         object 
ALTRADENOMINAZIONE         object 
ALTRADENOMTRASLITTERATA    object 
IDPROVINCIA                int64  
IDREGIONE                  object 
IDPREFETTURA               object 
STATO                      object 
SIGLAPROVINCIA             object 
FONTE                      object 
DATAULTIMOAGG              object 
COD_DENOM                  float64
dtype: object

In [4]:
date_columns = ['DATAISTITUZIONE', 'DATAULTIMOAGG']

def to_datetime(df, columns):
    for c in columns:
        df[c] = pd.to_datetime(df[c])
    return df

anpr_comuni = to_datetime(anpr_comuni, date_columns)

In [5]:
anpr_comuni.dtypes

ID                         int64         
DATAISTITUZIONE            datetime64[ns]
DATACESSAZIONE             object        
CODISTAT                   object        
CODCATASTALE               object        
DENOMINAZIONE_IT           object        
DENOMTRASLITTERATA         object        
ALTRADENOMINAZIONE         object        
ALTRADENOMTRASLITTERATA    object        
IDPROVINCIA                int64         
IDREGIONE                  object        
IDPREFETTURA               object        
STATO                      object        
SIGLAPROVINCIA             object        
FONTE                      object        
DATAULTIMOAGG              datetime64[ns]
COD_DENOM                  float64       
dtype: object

In [6]:
anpr_comuni.head(5)

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM
0,12560,1866-11-19,13/11/1924,028001,A001,ABANO,ABANO,NaN,NaN,28,05,NaN,C,PD,NaN,2016-06-17,28500.0
1,1,1924-11-14,31/12/9999,028001,A001,ABANO TERME,ABANO TERME,NaN,NaN,28,05,PD,A,PD,NaN,2016-06-17,NaN
2,3,1864-01-04,05/12/1928,001801,A003,ABBADIA ALPINA,ABBADIA ALPINA,NaN,NaN,1,01,NaN,C,TO,NaN,2017-11-11,NaN
3,5,1861-03-17,15/04/1992,015001,A004,ABBADIA CERRETO,ABBADIA CERRETO,NaN,NaN,15,03,NaN,C,MI,NaN,2016-06-17,NaN
4,4,1992-04-16,31/12/9999,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,NaN,NaN,98,03,LO,A,LO,NaN,2016-06-17,NaN


## Aggiungere colonna per individuare Comuni attivi

In [14]:
anpr_comuni['ATTIVO'] = anpr_comuni['DATACESSAZIONE'] ==  anpr_comuni['DATACESSAZIONE'].max()
anpr_comuni.shape

(19277, 18)

In [16]:
anpr_comuni.head()

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM,ATTIVO
0,12560,1866-11-19,13/11/1924,028001,A001,ABANO,ABANO,NaN,NaN,28,05,NaN,C,PD,NaN,2016-06-17,28500.0,False
1,1,1924-11-14,31/12/9999,028001,A001,ABANO TERME,ABANO TERME,NaN,NaN,28,05,PD,A,PD,NaN,2016-06-17,NaN,True
2,3,1864-01-04,05/12/1928,001801,A003,ABBADIA ALPINA,ABBADIA ALPINA,NaN,NaN,1,01,NaN,C,TO,NaN,2017-11-11,NaN,False
3,5,1861-03-17,15/04/1992,015001,A004,ABBADIA CERRETO,ABBADIA CERRETO,NaN,NaN,15,03,NaN,C,MI,NaN,2016-06-17,NaN,False
4,4,1992-04-16,31/12/9999,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,NaN,NaN,98,03,LO,A,LO,NaN,2016-06-17,NaN,True


In [15]:
# verificare con mirko se questa è la info che ci dice che il comune è "valido"
temp =anpr_comuni[anpr_comuni["DATACESSAZIONE"]=='31/12/9999']
temp.shape

(7954, 18)

In [ ]:
anpr_comuni.DATAISTITUZIONE.unique()

In [ ]:
anpr_comuni.DATAULTIMOAGG.unique()

In [ ]:
anpr_comuni.DATACESSAZIONE.isnull().sum()

In [ ]:
anpr_comuni[anpr_comuni.DENOMINAZIONE_IT=='TERRALBA']

In [ ]:
len(anpr_comuni)

In [ ]:
anpr_comuni.CODCATASTALE.nunique()

#### Codici ISTAT univoci

In [ ]:
anpr_comuni.CODISTAT.nunique()

## ISTAT: integrazione info a corollario



### Load elenco comuni

file Elenco-comuni-italiani.csv

In [17]:
istat_comuni_elenco = pd.read_csv("./data/Elenco-comuni-italiani.csv", dtype={'Codice Comune formato alfanumerico':'str'},
                                  encoding="ISO-8859-1", sep=";")
istat_comuni_elenco.rename(columns={"Codice Comune formato alfanumerico": 'CODISTAT'}, inplace=True)

In [18]:
istat_comuni_elenco.head(5) 

,Codice Regione,Codice Città Metropolitana,Codice Provincia (1),Progressivo del Comune (2),CODISTAT,Denominazione in italiano,Denominazione altra lingua,Codice Ripartizione Geografica,Ripartizione geografica,Denominazione regione,Denominazione Città metropolitana,Denominazione provincia,Flag Comune capoluogo di provincia,Sigla automobilistica,Codice Comune formato numerico,Codice Comune numerico con 110 province (dal 2010 al 2016),Codice Comune numerico con 107 province (dal 2006 al 2009),Codice Comune numerico con 103 province (dal 1995 al 2005),Codice Catastale del comune,Popolazione legale 2011 (09/10/2011),Codice NUTS1 2010,Codice NUTS2 2010 (3),Codice NUTS3 2010,Codice NUTS1 2006,Codice NUTS2 2006 (3),Codice NUTS3 2006
0,1.0,201,1.0,1.0,001001,Agliè,NaN,1.0,Nord-ovest,Piemonte,Torino,-,0.0,TO,1001.0,1001.0,1001.0,1001.0,A074,2.644,ITC,ITC1,ITC11,ITC,ITC1,ITC11
1,1.0,201,1.0,2.0,001002,Airasca,NaN,1.0,Nord-ovest,Piemonte,Torino,-,0.0,TO,1002.0,1002.0,1002.0,1002.0,A109,3.819,ITC,ITC1,ITC11,ITC,ITC1,ITC11
2,1.0,201,1.0,3.0,001003,Ala di Stura,NaN,1.0,Nord-ovest,Piemonte,Torino,-,0.0,TO,1003.0,1003.0,1003.0,1003.0,A117,462,ITC,ITC1,ITC11,ITC,ITC1,ITC11
3,1.0,201,1.0,4.0,001004,Albiano d'Ivrea,NaN,1.0,Nord-ovest,Piemonte,Torino,-,0.0,TO,1004.0,1004.0,1004.0,1004.0,A157,1.791,ITC,ITC1,ITC11,ITC,ITC1,ITC11
4,1.0,201,1.0,5.0,001005,Alice Superiore,NaN,1.0,Nord-ovest,Piemonte,Torino,-,0.0,TO,1005.0,1005.0,1005.0,1005.0,A199,701,ITC,ITC1,ITC11,ITC,ITC1,ITC11


In [19]:
final_0 = anpr_comuni.merge(istat_comuni_elenco, on='CODISTAT', how='inner')

In [20]:
len(final_0)

12376

### Load statistiche comuni

In [21]:
istat_comuni_stat = pd.read_excel("./data/Classificazioni statistiche_comuni_17_02_2018.xls", 
                                  dtype={'Codice Istat del Comune \n(formato alfanumerico)':'str', 
                                         'Codice Istat del Comune \n(formato numerico)': np.int64}
                    )


In [22]:
istat_comuni_stat.rename(columns={"Codice Istat del Comune \n(formato alfanumerico)": 'CODISTAT',
                                  "Codice Istat del Comune \n(formato numerico)":"CODISTAT_N"}, inplace=True)

In [23]:
istat_comuni_stat.dtypes

Codice Regione                                 int64  
CODISTAT                                       object 
CODISTAT_N                                     int64  
Denominazione in italiano                      object 
Denominazione altra lingua                     object 
Zona altimetrica                               int64  
Altitudine del centro (metri)                  int64  
Comune litoraneo                               int64  
Comune Montano                                 object 
Superficie territoriale (kmq) al 09/10/2011    float64
Grado di urbanizzazione                        int64  
dtype: object

In [24]:
istat_comuni_stat.head(3)

,Codice Regione,CODISTAT,CODISTAT_N,Denominazione in italiano,Denominazione altra lingua,Zona altimetrica,Altitudine del centro (metri),Comune litoraneo,Comune Montano,Superficie territoriale (kmq) al 09/10/2011,Grado di urbanizzazione
0,1,001001,1001,Agliè,NaN,3,315,0,NM,13.1464,2
1,1,001002,1002,Airasca,NaN,5,257,0,NM,15.7395,3
2,1,001003,1003,Ala di Stura,NaN,1,1080,0,T,46.3322,3


## Join ANPR/ISTAT lista/ISTAT stat 

In [25]:
final_1 = final_0.merge(istat_comuni_stat, on='CODISTAT', how='inner') #left

In [26]:
final_1.dtypes

ID                                                            int64         
DATAISTITUZIONE                                               datetime64[ns]
DATACESSAZIONE                                                object        
CODISTAT                                                      object        
CODCATASTALE                                                  object        
DENOMINAZIONE_IT                                              object        
DENOMTRASLITTERATA                                            object        
ALTRADENOMINAZIONE                                            object        
ALTRADENOMTRASLITTERATA                                       object        
IDPROVINCIA                                                   int64         
IDREGIONE                                                     object        
IDPREFETTURA                                                  object        
STATO                                                         object        

In [27]:
final_1.head(6)

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM,ATTIVO,Codice Regione_x,Codice Città Metropolitana,Codice Provincia (1),Progressivo del Comune (2),Denominazione in italiano_x,Denominazione altra lingua_x,Codice Ripartizione Geografica,Ripartizione geografica,Denominazione regione,Denominazione Città metropolitana,Denominazione provincia,Flag Comune capoluogo di provincia,Sigla automobilistica,Codice Comune formato numerico,Codice Comune numerico con 110 province (dal 2010 al 2016),Codice Comune numerico con 107 province (dal 2006 al 2009),Codice Comune numerico con 103 province (dal 1995 al 2005),Codice Catastale del comune,Popolazione legale 2011 (09/10/2011),Codice NUTS1 2010,Codice NUTS2 2010 (3),Codice NUTS3 2010,Codice NUTS1 2006,Codice NUTS2 2006 (3),Codice NUTS3 2006,Codice Regione_y,CODISTAT_N,Denominazione in italiano_y,Denominazione altra lingua_y,Zona altimetrica,Altitudine del centro (metri),Comune litoraneo,Comune Montano,Superficie territoriale (kmq) al 09/10/2011,Grado di urbanizzazione
0,12560,1866-11-19,13/11/1924,028001,A001,ABANO,ABANO,NaN,NaN,28,05,NaN,C,PD,NaN,2016-06-17,28500.0,False,5.0,-,28.0,1.0,Abano Terme,NaN,2.0,Nord-est,Veneto,-,Padova,0.0,PD,28001.0,28001.0,28001.0,28001.0,A001,19.349,ITH,ITH3,ITH36,ITD,ITD3,ITD36,5,28001,Abano Terme,NaN,5,14,0,NM,21.4084,2
1,1,1924-11-14,31/12/9999,028001,A001,ABANO TERME,ABANO TERME,NaN,NaN,28,05,PD,A,PD,NaN,2016-06-17,NaN,True,5.0,-,28.0,1.0,Abano Terme,NaN,2.0,Nord-est,Veneto,-,Padova,0.0,PD,28001.0,28001.0,28001.0,28001.0,A001,19.349,ITH,ITH3,ITH36,ITD,ITD3,ITD36,5,28001,Abano Terme,NaN,5,14,0,NM,21.4084,2
2,4,1992-04-16,31/12/9999,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,NaN,NaN,98,03,LO,A,LO,NaN,2016-06-17,NaN,True,3.0,-,98.0,1.0,Abbadia Cerreto,NaN,1.0,Nord-ovest,Lombardia,-,Lodi,0.0,LO,98001.0,98001.0,98001.0,98001.0,A004,297,ITC,ITC4,ITC49,ITC,ITC4,ITC49,3,98001,Abbadia Cerreto,NaN,5,64,0,NM,6.1987,3
3,7,1992-04-16,31/12/9999,097001,A005,ABBADIA LARIANA,ABBADIA LARIANA,NaN,NaN,97,03,LC,A,LC,NaN,2016-06-17,NaN,True,3.0,-,97.0,1.0,Abbadia Lariana,NaN,1.0,Nord-ovest,Lombardia,-,Lecco,0.0,LC,97001.0,97001.0,97001.0,97001.0,A005,3.249,ITC,ITC4,ITC43,ITC,ITC4,ITC43,3,97001,Abbadia Lariana,NaN,1,204,0,T,16.6727,2
4,8,1867-07-22,31/12/9999,052001,A006,ABBADIA SAN SALVATORE,ABBADIA SAN SALVATORE,NaN,NaN,52,09,SI,A,SI,NaN,2016-06-17,NaN,True,9.0,-,52.0,1.0,Abbadia San Salvatore,NaN,3.0,Centro,Toscana,-,Siena,0.0,SI,52001.0,52001.0,52001.0,52001.0,A006,6.557,ITI,ITI1,ITI19,ITE,ITE1,ITE19,9,52001,Abbadia San Salvatore,NaN,1,822,0,T,58.9942,2
5,10,1974-08-20,31/12/9999,095001,A007,ABBASANTA,ABBASANTA,NaN,NaN,95,20,OR,A,OR,NaN,2016-06-17,NaN,True,20.0,-,95.0,1.0,Abbasanta,NaN,5.0,Isole,Sardegna,-,Oristano,0.0,OR,95001.0,95001.0,95001.0,95001.0,A007,2.828,ITG,ITG2,ITG28,ITG,ITG2,ITG28,20,95001,Abbasanta,NaN,3,315,0,T,39.8475,3


In [28]:
drop = ['Codice Regione_x',
        'Codice Provincia (1)',
        'Progressivo del Comune (2)',
        'Denominazione in italiano_x',
        'Denominazione altra lingua_x',
        'Codice Catastale del comune',
        'Codice Regione_y',
        'Denominazione in italiano_y',
        'Denominazione altra lingua_y']


# drop by Name
final_1 = final_1.drop(drop, axis=1)

In [29]:
len(final_1)

12375

### Load geojson dei comuni al 2016 da fonte ISTAT

Il codice istat è numerico, qui ci sono solo i record dei comuni al 2016


In [30]:
fname = "./data/Comuni_2016.geojson"
comuni_geo = gpd.read_file(fname)
#aggiungo latitudine e longitudine del centroide
comuni_geo["lat"] = comuni_geo.geometry.centroid.y
comuni_geo["long"] = comuni_geo.geometry.centroid.x

In [31]:
#check
len(comuni_geo)
comuni_geo[comuni_geo.COMUNE=='Terralba']

7998

COD_REG  COD_CM  COD_PRO  PRO_COM    COMUNE NOME_TED  FLAG_CM    SHAPE_Leng    SHAPE_Area  \
7859  20       0       95       95065    Terralba           2        51724.608693  4.979083e+07   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
#comuni_geo.plot(column='COMUNE', categorical=True, legend=True, figsize=(18, 10));

## JOIN Geo 

In [35]:
full = final_1.merge(comuni_geo[['PRO_COM','geometry','lat','long']], left_on='CODISTAT_N', right_on='PRO_COM', how='inner')

In [36]:
len(full)

12113

In [37]:
full.head(10)

ID DATAISTITUZIONE DATACESSAZIONE CODISTAT CODCATASTALE       DENOMINAZIONE_IT     DENOMTRASLITTERATA ALTRADENOMINAZIONE ALTRADENOMTRASLITTERATA  IDPROVINCIA IDREGIONE IDPREFETTURA STATO SIGLAPROVINCIA FONTE DATAULTIMOAGG  COD_DENOM  ATTIVO Codice Città Metropolitana  Codice Ripartizione Geografica Ripartizione geografica Denominazione regione Denominazione Città metropolitana Denominazione provincia  Flag Comune capoluogo di provincia Sigla automobilistica  Codice Comune formato numerico  Codice Comune numerico con 110 province (dal 2010 al 2016)  Codice Comune numerico con 107 province (dal 2006 al 2009)  Codice Comune numerico con 103 province (dal 1995 al 2005) Popolazione legale 2011 (09/10/2011) Codice NUTS1 2010 Codice NUTS2 2010 (3)  Codice NUTS3 2010 Codice NUTS1 2006 Codice NUTS2 2006 (3) Codice NUTS3 2006  CODISTAT_N  Zona altimetrica  Altitudine del centro (metri)  Comune litoraneo Comune Montano  Superficie territoriale (kmq) al 09/10/2011  Grado di urbanizzazione  PRO_COM  \
0  12560 1866-11-19       13/11/1924     028001   A001         ABANO                  ABANO                  NaN                NaN                     28           05        NaN          C     PD             NaN  2016-06-17     28500.0    False   -                          2.0                             Nord-est                Veneto                -                                 Padova                  0.0                                 PD                    28001.0                         28001.0                                                     28001.0                                                     28001.0                                                     19.349                               ITH               ITH3                   ITH36             ITD               ITD3                  ITD36             28001       5                 14                             0                 NM             21.4084                                      2                        28001     
1  1     1924-11-14       31/12/9999     028001   A001         ABANO TERME            ABANO TERME            NaN                NaN                     28           05        PD           A     PD             NaN  2016-06-17    NaN         True    -                          2.0                             Nord-est                Veneto                -                                 Padova                  0.0                                 PD                    28001.0                         28001.0                                                     28001.0                                                     28001.0                                                     19.349                               ITH               ITH3                   ITH36             ITD               ITD3                  ITD36             28001       5                 14                             0                 NM             21.4084                                      2                        28001     
2  4     1992-04-16       31/12/9999     098001   A004         ABBADIA CERRETO        ABBADIA CERRETO        NaN                NaN                     98           03        LO           A     LO             NaN  2016-06-17    NaN         True    -                          1.0                             Nord-ovest              Lombardia             -                                 Lodi                    0.0                                 LO                    98001.0                         98001.0                                                     98001.0                                                     98001.0                                                     297                                  ITC               ITC4                   ITC49             ITC               ITC4                  ITC49             98001       5                 64                             0                 NM             6.1987                                       3 

In [38]:
full.geometry.isnull().sum()

0

In [39]:
full.dtypes

ID                                                            int64         
DATAISTITUZIONE                                               datetime64[ns]
DATACESSAZIONE                                                object        
CODISTAT                                                      object        
CODCATASTALE                                                  object        
DENOMINAZIONE_IT                                              object        
DENOMTRASLITTERATA                                            object        
ALTRADENOMINAZIONE                                            object        
ALTRADENOMTRASLITTERATA                                       object        
IDPROVINCIA                                                   int64         
IDREGIONE                                                     object        
IDPREFETTURA                                                  object        
STATO                                                         object        

### Save final dataframe 

In [40]:
full.to_csv("./data/full.csv", index=False, sep=";" )